In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from pathlib import Path

## 1. Load embeddings đã có

In [ ]:
# Load embeddings
image_embeddings = np.load('/kaggle/input/your-embeddings-dataset/medsiglip_image_embeddings.npy')
text_embeddings = np.load('/kaggle/input/your-embeddings-dataset/medsiglip_text_embeddings.npy')

print(f"Image embeddings shape: {image_embeddings.shape}")
print(f"Text embeddings shape: {text_embeddings.shape}")

## 2. Tạo danh sách đường dẫn ảnh

In [ ]:
# Đường dẫn đến thư mục ảnh
image_dir = '/kaggle/input/chest-x-ray/images/images_normalized'

# Lấy tất cả các file ảnh .png
image_paths = sorted([str(p) for p in Path(image_dir).glob('*.png')])
print(f"Tổng số ảnh: {len(image_paths)}")
print(f"Ví dụ ảnh đầu tiên: {image_paths[0]}")

## 3. Hàm tính cosine similarity

In [ ]:
def cosine_similarity(query_embedding, database_embeddings):
    """
    Tính cosine similarity giữa query và database embeddings
    
    Args:
        query_embedding: (1152,) hoặc (1, 1152)
        database_embeddings: (N, 1152)
    
    Returns:
        similarities: (N,) array of cosine similarities
    """
    # Normalize embeddings
    query_norm = query_embedding / np.linalg.norm(query_embedding)
    db_norm = database_embeddings / np.linalg.norm(database_embeddings, axis=1, keepdims=True)
    
    # Tính cosine similarity
    similarities = np.dot(db_norm, query_norm.flatten())
    
    return similarities

## 4. Hàm retrieve top-k ảnh tương tự

In [ ]:
def retrieve_similar_images(query_idx, image_embeddings, image_paths, top_k=10):
    """
    Tìm top-k ảnh tương tự nhất với query image
    
    Args:
        query_idx: index của ảnh query
        image_embeddings: (N, 1152) embeddings của tất cả ảnh
        image_paths: list đường dẫn ảnh
        top_k: số lượng ảnh tương tự cần trả về
    
    Returns:
        top_indices: indices của top-k ảnh
        top_similarities: cosine similarities tương ứng
        top_paths: đường dẫn của top-k ảnh
    """
    # Lấy embedding của query image
    query_embedding = image_embeddings[query_idx]
    
    # Tính similarity với tất cả ảnh
    similarities = cosine_similarity(query_embedding, image_embeddings)
    
    # Lấy top-k indices (bao gồm cả chính nó)
    top_indices = np.argsort(similarities)[::-1][:top_k]
    top_similarities = similarities[top_indices]
    top_paths = [image_paths[idx] for idx in top_indices]
    
    return top_indices, top_similarities, top_paths

## 5. Hàm hiển thị kết quả

In [ ]:
def display_retrieval_results(query_path, top_paths, top_similarities, top_k=10):
    """
    Hiển thị query image và top-k ảnh tương tự
    """
    # Tính số hàng cần thiết (query + top_k)
    n_cols = 5
    n_rows = (top_k + n_cols - 1) // n_cols + 1  # +1 cho query image
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 4*n_rows))
    axes = axes.flatten()
    
    # Hiển thị query image
    query_img = Image.open(query_path)
    axes[0].imshow(query_img, cmap='gray')
    axes[0].set_title(f"QUERY IMAGE\n{os.path.basename(query_path)}", 
                      fontsize=12, fontweight='bold', color='red')
    axes[0].axis('off')
    
    # Ẩn các ô trống trong hàng đầu
    for i in range(1, n_cols):
        axes[i].axis('off')
    
    # Hiển thị top-k ảnh tương tự
    for i, (path, sim) in enumerate(zip(top_paths, top_similarities)):
        idx = n_cols + i  # Bắt đầu từ hàng thứ 2
        if idx < len(axes):
            img = Image.open(path)
            axes[idx].imshow(img, cmap='gray')
            axes[idx].set_title(f"Rank {i+1}\nSimilarity: {sim:.4f}\n{os.path.basename(path)}", 
                               fontsize=10)
            axes[idx].axis('off')
    
    # Ẩn các ô còn lại
    for i in range(n_cols + top_k, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # In thông tin chi tiết
    print("\n" + "="*80)
    print("RETRIEVAL RESULTS")
    print("="*80)
    print(f"Query: {os.path.basename(query_path)}\n")
    for i, (path, sim) in enumerate(zip(top_paths, top_similarities)):
        print(f"Rank {i+1:2d} | Similarity: {sim:.6f} | {os.path.basename(path)}")

## 6. TEST: Retrieval với ảnh cụ thể

In [ ]:
# Đường dẫn ảnh query
query_path = '/kaggle/input/chest-x-ray/images/images_normalized/1000_IM-0003-1001.dcm.png'

# Tìm index của ảnh query trong danh sách
try:
    query_idx = image_paths.index(query_path)
    print(f"Found query image at index: {query_idx}")
except ValueError:
    print(f"ERROR: Query image not found in image_paths!")
    print(f"Query path: {query_path}")
    print(f"\nKiểm tra các ảnh có sẵn:")
    for i, p in enumerate(image_paths[:5]):
        print(f"  {i}: {p}")
    query_idx = None

In [ ]:
# Nếu tìm thấy ảnh query, thực hiện retrieval
if query_idx is not None:
    # Retrieve top 10 ảnh tương tự (bao gồm cả chính nó)
    top_indices, top_similarities, top_paths = retrieve_similar_images(
        query_idx=query_idx,
        image_embeddings=image_embeddings,
        image_paths=image_paths,
        top_k=10
    )
    
    # Hiển thị kết quả
    display_retrieval_results(query_path, top_paths, top_similarities, top_k=10)

## 7. TEST với nhiều ảnh khác nhau

In [ ]:
# Test với 3 ảnh random
np.random.seed(42)
random_indices = np.random.choice(len(image_paths), size=3, replace=False)

for idx in random_indices:
    print(f"\n{'='*80}")
    print(f"Testing with image index: {idx}")
    print(f"{'='*80}\n")
    
    top_indices, top_similarities, top_paths = retrieve_similar_images(
        query_idx=idx,
        image_embeddings=image_embeddings,
        image_paths=image_paths,
        top_k=10
    )
    
    display_retrieval_results(image_paths[idx], top_paths, top_similarities, top_k=10)

## 8. Phân tích độ chính xác của retrieval

In [ ]:
# Tính distribution của similarity scores
def analyze_retrieval_quality(image_embeddings, n_samples=100):
    """
    Phân tích chất lượng retrieval trên nhiều samples
    """
    np.random.seed(42)
    sample_indices = np.random.choice(len(image_embeddings), size=n_samples, replace=False)
    
    all_top1_sims = []
    all_top5_sims = []
    all_top10_sims = []
    
    for idx in sample_indices:
        query_embedding = image_embeddings[idx]
        similarities = cosine_similarity(query_embedding, image_embeddings)
        
        # Lấy top-k similarities (bỏ chính nó - index 0)
        top_sims = np.sort(similarities)[::-1]
        
        all_top1_sims.append(top_sims[1])  # Top-1 (bỏ chính nó)
        all_top5_sims.extend(top_sims[1:6])  # Top 2-6
        all_top10_sims.extend(top_sims[1:11])  # Top 2-11
    
    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(18, 4))
    
    axes[0].hist(all_top1_sims, bins=50, edgecolor='black')
    axes[0].set_title(f'Top-1 Similarity Distribution\nMean: {np.mean(all_top1_sims):.4f}')
    axes[0].set_xlabel('Cosine Similarity')
    axes[0].set_ylabel('Frequency')
    
    axes[1].hist(all_top5_sims, bins=50, edgecolor='black', color='orange')
    axes[1].set_title(f'Top-5 Similarity Distribution\nMean: {np.mean(all_top5_sims):.4f}')
    axes[1].set_xlabel('Cosine Similarity')
    axes[1].set_ylabel('Frequency')
    
    axes[2].hist(all_top10_sims, bins=50, edgecolor='black', color='green')
    axes[2].set_title(f'Top-10 Similarity Distribution\nMean: {np.mean(all_top10_sims):.4f}')
    axes[2].set_xlabel('Cosine Similarity')
    axes[2].set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nStatistics over {n_samples} samples:")
    print(f"Top-1 avg similarity: {np.mean(all_top1_sims):.4f} ± {np.std(all_top1_sims):.4f}")
    print(f"Top-5 avg similarity: {np.mean(all_top5_sims):.4f} ± {np.std(all_top5_sims):.4f}")
    print(f"Top-10 avg similarity: {np.mean(all_top10_sims):.4f} ± {np.std(all_top10_sims):.4f}")

# Chạy phân tích
analyze_retrieval_quality(image_embeddings, n_samples=100)